# Data Preprocessing
- Environment setup
- Load both 2017, 2018 original and improved CICIDS datasets
- Combine all the protions of dataset into one dataframe
- Rename Columns
- Remove duplicate and missing value rows
- Reclassify the data labels to ['Benign', 'Attack']
- Encode Labels
- Remove duplicate column for 2017 original dataset - Fwd Header Length.1
- Extract features by random forest (feature importance)
- Generate data only contains selected features
- Fix Data Types

### Environment setup

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!wget https://downloads.rclone.org/v1.63.0/rclone-v1.63.0-linux-amd64.deb
!apt install ./rclone-v1.63.0-linux-amd64.deb
!rclone config
!sudo apt-get -y install fuse3

--2023-07-27 08:26:20--  https://downloads.rclone.org/v1.63.0/rclone-v1.63.0-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone.org)|95.217.6.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18373062 (18M) [application/vnd.debian.binary-package]
Saving to: ‘rclone-v1.63.0-linux-amd64.deb’

rclone-v1.63.0-linu 100%[===================>]  17.52M  12.0MB/s    in 1.5s    

2023-07-27 08:26:23 (12.0 MB/s) - ‘rclone-v1.63.0-linux-amd64.deb’ saved [18373062/18373062]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.63.0-linux-amd64.deb'
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 0 B/18.4 MB of archives.
After this operation, 56.7 MB of additional disk space will be used.
Get:

In [ ]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

nohup: appending output to 'nohup.out'


### Load both 2017, 2018 original and improved CICIDS datasets

In [ ]:
# Import Necessary Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob

In [ ]:
df2017_original_files = glob.glob('2017/*.csv')

for file in df2017_original_files:
    df = pd.read_csv(file)
    df.to_parquet(f'{file[:-4]}.parquet', compression=None)

In [ ]:
df2017_corrected_files = glob.glob('improved_2017/*.csv')

for file in df2017_corrected_files:
    df = pd.read_csv(file)
    df.to_parquet(f'{file[:-4]}.parquet', compression=None)

In [ ]:
df2018_original_files = glob.glob('2018/*.csv')

for file in df2018_original_files:
    df = pd.read_csv(file, low_memory=False)
    df.to_parquet(f'{file[:-4]}.parquet', compression=None)

Experiment for time reading file (check methods if improved)

In [ ]:
import time
start = time.time()

pd.read_csv('2018/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv')

print(f'time for reading csv file: {time.time() - start}')

In [ ]:
start = time.time()

pd.read_parquet('2018/Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet', engine="fastparquet")

print(f'time for reading csv file: {time.time() - start}')



time for reading csv file: 2.0231380462646484


### read first dataset

In [ ]:
# df2017_original = pd.read_parquet('/content/onedrive/2017_original/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.parquet')
df2018_original = pd.read_parquet('../content/onedrive/2018_original/Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet')
# df2017_corrected = pd.read_parquet('/content/onedrive/Improved/CICIDS2017_improved/friday.parquet')
# df2018_corrected = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Friday-02-03-2018.parquet')


In [ ]:
row_2017, col_2017 = df2017_original.shape
print(f'There are {col_2017} columns, {row_2017} rows in CICIDS2017')
# row_2018, col_2018 = df2018_original.shape
# print(f'There are {col_2018} columns, {row_2018} rows in CICIDS2018')
row_2017, col_2017 = df2017_corrected.shape
print(f'There are {col_2017} columns, {row_2017} rows in CICIDS2017')
# row_2018, col_2018 = df2018_corrected.shape
# print(f'There are {col_2018} columns, {row_2018} rows in CICIDS2018')

In [ ]:
df2017_original.head(5)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [ ]:
df2017_corrected.head(5)

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,...,343,1.610540e+07,4.988048e+05,16399772,15375229,-1,-1,112740690,BENIGN,-1
1,2,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,...,285,1.610543e+07,4.987937e+05,16399782,15375263,-1,-1,112740560,BENIGN,-1
2,3,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,545,...,19,1.221036e+07,6.935824e+06,20757030,5504997,-1,-1,0,BENIGN,-1
3,4,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,...,16,1.319764e+07,5.826905e+06,19776791,5817470,-1,-1,0,BENIGN,-1
4,5,192.168.10.25-17.253.14.125-123-123-17,192.168.10.25,123,17.253.14.125,123,17,2017-07-07 12:00:42.430758,66966070,6,...,1968172,6.497443e+07,0.000000e+00,64974431,64974431,-1,-1,0,BENIGN,-1


In [ ]:
df2018_original.head(5)

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773,202,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0,38,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527,385,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141,517,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [ ]:
df2018_corrected.head(5)

In [ ]:
print(f'2017 original columns: {df2017_original.columns}')
print(f'2017 corrected columns: {df2017_corrected.columns}')
print(f'2018 original columns: {df2018_original.columns}')
# print(f'2018 corrected columns: {df2018_corrected.columns}')

2017 original columns: Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Pac

In [ ]:
df2017_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225745 entries, 0 to 225744
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             225745 non-null  int64  
 1    Flow Duration                225745 non-null  int64  
 2    Total Fwd Packets            225745 non-null  int64  
 3    Total Backward Packets       225745 non-null  int64  
 4   Total Length of Fwd Packets   225745 non-null  int64  
 5    Total Length of Bwd Packets  225745 non-null  int64  
 6    Fwd Packet Length Max        225745 non-null  int64  
 7    Fwd Packet Length Min        225745 non-null  int64  
 8    Fwd Packet Length Mean       225745 non-null  float64
 9    Fwd Packet Length Std        225745 non-null  float64
 10  Bwd Packet Length Max         225745 non-null  int64  
 11   Bwd Packet Length Min        225745 non-null  int64  
 12   Bwd Packet Length Mean       225745 non-nul

### Combine all the protions of dataset into one dataframe

In [ ]:
d0_2017_o = df2017_original #the first row portion is already imported so we will just copy that
d1_2017_o = pd.read_parquet('../resources/2017_original/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.parquet')
d2_2017_o = pd.read_parquet('../resources/2017_original/Friday-WorkingHours-Morning.pcap_ISCX.parquet')
d3_2017_o = pd.read_parquet('../resources/2017_original/Monday-WorkingHours.pcap_ISCX.parquet')
d4_2017_o = pd.read_parquet('../resources/2017_original/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.parquet')
d5_2017_o = pd.read_parquet('../resources/2017_original/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.parquet')
d6_2017_o = pd.read_parquet('../resources/2017_original/Tuesday-WorkingHours.pcap_ISCX.parquet')
d7_2017_o = pd.read_parquet('../resources/2017_original/Wednesday-workingHours.pcap_ISCX.parquet')

In [ ]:
d0_2017_c = df2017_corrected #the first row portion is already imported so we will just copy that
d1_2017_c = pd.read_parquet('/content/onedrive/Improved/CICIDS2017_improved/monday.parquet')
d2_2017_c = pd.read_parquet('/content/onedrive/Improved/CICIDS2017_improved/thursday.parquet')
d3_2017_c = pd.read_parquet('/content/onedrive/Improved/CICIDS2017_improved/tuesday.parquet')
d4_2017_c = pd.read_parquet('/content/onedrive/Improved/CICIDS2017_improved/wednesday.parquet')

In [ ]:
d0_2018_o = df2018_original #the first row portion is already imported so we will just copy that
del df2018_original
d1_2018_o = pd.read_parquet('/content/onedrive/2018_original/Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet')
d2_2018_o = pd.read_parquet('/content/onedrive/2018_original/Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet')
d3_2018_o = pd.read_parquet('/content/onedrive/2018_original/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.parquet')
d3_2018_o.drop(columns=['Flow ID', 'Src IP', 'Src Port', 'Dst IP'], axis=1,inplace=True)
d4_2018_o = pd.read_parquet('/content/onedrive/2018_original/Thursday-01-03-2018_TrafficForML_CICFlowMeter.parquet')
d5_2018_o = pd.read_parquet('/content/onedrive/2018_original/Thursday-15-02-2018_TrafficForML_CICFlowMeter.parquet')
d6_2018_o = pd.read_parquet('/content/onedrive/2018_original/Thursday-22-02-2018_TrafficForML_CICFlowMeter.parquet')
d7_2018_o = pd.read_parquet('/content/onedrive/2018_original/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.parquet')
d8_2018_o = pd.read_parquet('/content/onedrive/2018_original/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.parquet')
d9_2018_o = pd.read_parquet('/content/onedrive/2018_original/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.parquet')

In [ ]:
columns = d0_2017_o.columns

for c in columns:
    if c in d0_2017_o.columns and c in d1_2017_o.columns and c in d2_2017_o.columns and c in d3_2017_o.columns and c in d4_2017_o.columns and c in d5_2017_o.columns and c in d6_2017_o.columns and c in d7_2017_o.columns:
        dtype0 = d0_2017_o[c].dtype
        dtype1 = d1_2017_o[c].dtype
        dtype2 = d2_2017_o[c].dtype
        dtype3 = d3_2017_o[c].dtype
        dtype4 = d4_2017_o[c].dtype
        dtype5 = d5_2017_o[c].dtype
        dtype6 = d6_2017_o[c].dtype
        dtype7 = d7_2017_o[c].dtype

        if dtype0 == dtype1 == dtype2 == dtype3 == dtype4 == dtype5 ==dtype6 == dtype7:
            print("The data types of '{}' in both dataframes are the same: {}".format(c, dtype1))
        else:
            print("The data types of '{}' in both dataframes are different.".format(c))
    else:
        print("The column '{}' does not exist in one or both dataframes.".format(c))


In [ ]:
# Combining the datasets
df2017_original = pd.concat([d0_2017_o, d1_2017_o, d2_2017_o, d3_2017_o, d4_2017_o, d5_2017_o, d6_2017_o, d7_2017_o], ignore_index=True)
df2017_corrected = pd.concat([d0_2017_c, d1_2017_c, d2_2017_c, d3_2017_c, d4_2017_c], ignore_index=True)
df2018_original = pd.concat([d0_2018_o, d1_2018_o, d2_2018_o, d3_2018_o, d4_2018_o, d5_2018_o, d6_2018_o, d7_2018_o, d8_2018_o, d9_2018_o], ignore_index=True)
# df2018_corrected = pd.concat([d0_2018_c, d1_2018_c, d2_2018_c, d3_2018_c, d4_2018_c, d5_2018_c, d6_2018_c, d7_2018_c, d8_2018_c, d9_2018_c], ignore_index=True)

In [ ]:
print(f'df2017_original.shape : {df2017_original.shape }')

df2017_original.shape : (2830743, 79)


In [ ]:
# print(f'df2017_corrected.shape : {df2017_corrected.shape }')
print(f'df2018_original.shape : {df2018_original.shape }')
# print(f'df2018_corrected.shape : {df2018_corrected.shape }')


In [ ]:
print(f'df2018_corrected.shape : {df2018_corrected.shape }')

df2018_corrected.shape : (63195145, 91)


In [ ]:
print(f'df2018_corrected.shape : {df2018_corrected.shape }')


df2018_corrected.shape : (63195088, 91)


### Rename Columns

In [ ]:
formatted_data = {' Destination Port': 'Destination Port',
                  ' Flow Duration': 'Flow Duration',
                  ' Total Fwd Packets': 'Total Fwd Packets',
                  ' Total Backward Packets': 'Total Backward Packets',
                  'Total Length of Fwd Packets': 'Total Length of Fwd Packets',
                  ' Total Length of Bwd Packets': 'Total Length of Bwd Packets',
                  ' Fwd Packet Length Max': 'Fwd Packet Length Max',
                  ' Fwd Packet Length Min': 'Fwd Packet Length Min',
                  ' Fwd Packet Length Mean': 'Fwd Packet Length Mean',
                  ' Fwd Packet Length Std': 'Fwd Packet Length Std',
                  'Bwd Packet Length Max': 'Bwd Packet Length Max',
                  ' Bwd Packet Length Min': 'Bwd Packet Length Min',
                  ' Bwd Packet Length Mean': 'Bwd Packet Length Mean',
                  ' Bwd Packet Length Std': 'Bwd Packet Length Std',
                  'Flow Bytes/s': 'Flow Bytes/s',
                  ' Flow Packets/s': 'Flow Packets/s',
                  ' Flow IAT Mean': 'Flow IAT Mean',
                  ' Flow IAT Std': 'Flow IAT Std',
                  ' Flow IAT Max': 'Flow IAT Max',
                  ' Flow IAT Min': 'Flow IAT Min',
                  'Fwd IAT Total': 'Fwd IAT Total',
                  ' Fwd IAT Mean': 'Fwd IAT Mean',
                  ' Fwd IAT Std': 'Fwd IAT Std',
                  ' Fwd IAT Max': 'Fwd IAT Max',
                  ' Fwd IAT Min': 'Fwd IAT Min',
                  'Bwd IAT Total': 'Bwd IAT Total',
                  ' Bwd IAT Mean': 'Bwd IAT Mean',
                  ' Bwd IAT Std': 'Bwd IAT Std',
                  ' Bwd IAT Max': 'Bwd IAT Max',
                  ' Bwd IAT Min': 'Bwd IAT Min',
                  'Fwd PSH Flags': 'Fwd PSH Flags',
                  ' Bwd PSH Flags': 'Bwd PSH Flags',
                  ' Fwd URG Flags': 'Fwd URG Flags',
                  ' Bwd URG Flags': 'Bwd URG Flags',
                  ' Fwd Header Length': 'Fwd Header Length',
                  ' Bwd Header Length': 'Bwd Header Length',
                  'Fwd Packets/s': 'Fwd Packets/s',
                  ' Bwd Packets/s': 'Bwd Packets/s',
                  ' Min Packet Length': 'Min Packet Length',
                  ' Max Packet Length': 'Max Packet Length',
                  ' Packet Length Mean': 'Packet Length Mean',
                  ' Packet Length Std': 'Packet Length Std',
                  ' Packet Length Variance': 'Packet Length Variance',
                  'FIN Flag Count': 'FIN Flag Count',
                  ' SYN Flag Count': 'SYN Flag Count',
                  ' RST Flag Count': 'RST Flag Count',
                  ' PSH Flag Count': 'PSH Flag Count',
                  ' ACK Flag Count': 'ACK Flag Count',
                  ' URG Flag Count': 'URG Flag Count',
                  ' CWE Flag Count': 'CWE Flag Count',
                  ' ECE Flag Count': 'ECE Flag Count',
                  ' Down/Up Ratio': 'Down/Up Ratio',
                  ' Average Packet Size': 'Average Packet Size',
                  ' Avg Fwd Segment Size': 'Avg Fwd Segment Size',
                  ' Avg Bwd Segment Size': 'Avg Bwd Segment Size',
                  ' Fwd Header Length.1': 'Fwd Header Length.1',
                  'Fwd Avg Bytes/Bulk': 'Fwd Avg Bytes/Bulk',
                  ' Fwd Avg Packets/Bulk': 'Fwd Avg Packets/Bulk',
                  ' Fwd Avg Bulk Rate': 'Fwd Avg Bulk Rate',
                  ' Bwd Avg Bytes/Bulk': 'Bwd Avg Bytes/Bulk',
                  ' Bwd Avg Packets/Bulk': 'Bwd Avg Packets/Bulk',
                  'Bwd Avg Bulk Rate': 'Bwd Avg Bulk Rate',
                  'Subflow Fwd Packets': 'Subflow Fwd Packets',
                  ' Subflow Fwd Bytes': 'Subflow Fwd Bytes',
                  ' Subflow Bwd Packets': 'Subflow Bwd Packets',
                  ' Subflow Bwd Bytes': 'Subflow Bwd Bytes',
                  'Init_Win_bytes_forward': 'Init_Win_bytes_forward',
                  ' Init_Win_bytes_backward': 'Init_Win_bytes_backward',
                  ' act_data_pkt_fwd': 'act_data_pkt_fwd',
                  ' min_seg_size_forward': 'min_seg_size_forward',
                  'Active Mean': 'Active Mean',
                  ' Active Std': 'Active Std',
                  ' Active Max': 'Active Max',
                  ' Active Min': 'Active Min',
                  'Idle Mean': 'Idle Mean',
                  ' Idle Std': 'Idle Std',
                  ' Idle Max': 'Idle Max',
                  ' Idle Min': 'Idle Min',
                  ' Label': 'Label'
}

In [ ]:
#rename dataset
df2017_original.rename(columns=formatted_data, inplace=True)

In [ ]:
formatted_data = {
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packet': 'Total Fwd Packets',
    'Total Bwd packets': 'Total Backward Packets',
    'Total Length of Fwd Packet': 'Total Length of Fwd Packets',
    'Total Length of Bwd Packet': 'Total Length of Bwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    'Bwd PSH Flags': 'Bwd PSH Flags',
    'Fwd URG Flags': 'Fwd URG Flags',
    'Bwd URG Flags': 'Bwd URG Flags',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Packet Length Min': 'Min Packet Length',
    'Packet Length Max': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'SYN Flag Count': 'SYN Flag Count',
    'RST Flag Count': 'RST Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'URG Flag Count': 'URG Flag Count',
    'CWR Flag Count': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Count',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Average Packet Size': 'Average Packet Size',
    'Fwd Segment Size Avg': 'Avg Fwd Segment Size',
    'Bwd Segment Size Avg': 'Avg Bwd Segment Size',
    'Fwd Bytes/Bulk Avg': 'Fwd Avg Bytes/Bulk',
    'Fwd Packet/Bulk Avg': 'Fwd Avg Packets/Bulk',
    'Fwd Bulk Rate Avg': 'Fwd Avg Bulk Rate',
    'Bwd Bytes/Bulk Avg': 'Bwd Avg Bytes/Bulk',
    'Bwd Packet/Bulk Avg': 'Bwd Avg Packets/Bulk',
    'Bwd Bulk Rate Avg': 'Bwd Avg Bulk Rate',
    'Subflow Fwd Packets': 'Subflow Fwd Packets',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Subflow Bwd Packets': 'Subflow Bwd Packets',
    'Subflow Bwd Bytes': 'Subflow Bwd Bytes',
    'FWD Init Win Bytes': 'Init_Win_bytes_forward',
    'Bwd Init Win Bytes': 'Init_Win_bytes_backward',
    'Fwd Act Data Pkts': 'act_data_pkt_fwd',
    'Fwd Seg Size Min': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}

In [ ]:
df2017_corrected.rename(columns=formatted_data, inplace=True)

In [ ]:
formatted_data = {
    'Dst Port': 'Destination Port',
    'Flow Duration': 'Flow Duration',
    'Tot Fwd Pkts': 'Total Fwd Packets',
    'Tot Bwd Pkts': 'Total Backward Packets',
    'TotLen Fwd Pkts': 'Total Length of Fwd Packets',
    'TotLen Bwd Pkts': 'Total Length of Bwd Packets',
    'Fwd Pkt Len Max': 'Fwd Packet Length Max',
    'Fwd Pkt Len Min': 'Fwd Packet Length Min',
    'Fwd Pkt Len Mean': 'Fwd Packet Length Mean',
    'Fwd Pkt Len Std': 'Fwd Packet Length Std',
    'Bwd Pkt Len Max': 'Bwd Packet Length Max',
    'Bwd Pkt Len Min': 'Bwd Packet Length Min',
    'Bwd Pkt Len Mean': 'Bwd Packet Length Mean',
    'Bwd Pkt Len Std': 'Bwd Packet Length Std',
    'Flow Byts/s': 'Flow Bytes/s',
    'Flow Pkts/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Tot': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Tot': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    'Bwd PSH Flags': 'Bwd PSH Flags',
    'Fwd URG Flags': 'Fwd URG Flags',
    'Bwd URG Flags': 'Bwd URG Flags',
    'Fwd Header Len': 'Fwd Header Length',
    'Bwd Header Len': 'Bwd Header Length',
    'Fwd Pkts/s': 'Fwd Packets/s',
    'Bwd Pkts/s': 'Bwd Packets/s',
    'Pkt Len Min': 'Min Packet Length',
    'Pkt Len Max': 'Max Packet Length',
    'Pkt Len Mean': 'Packet Length Mean',
    'Pkt Len Std': 'Packet Length Std',
    'Pkt Len Var': 'Packet Length Variance',
    'FIN Flag Cnt': 'FIN Flag Count',
    'SYN Flag Cnt': 'SYN Flag Count',
    'RST Flag Cnt': 'RST Flag Count',
    'PSH Flag Cnt': 'PSH Flag Count',
    'ACK Flag Cnt': 'ACK Flag Count',
    'URG Flag Cnt': 'URG Flag Count',
    'CWE Flag Count': 'CWE Flag Count',
    'ECE Flag Cnt': 'ECE Flag Count',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Pkt Size Avg': 'Average Packet Size',
    'Fwd Seg Size Avg': 'Avg Fwd Segment Size',
    'Bwd Seg Size Avg': 'Avg Bwd Segment Size',
    'Fwd Byts/b Avg': 'Fwd Avg Bytes/Bulk',
    'Fwd Pkts/b Avg': 'Fwd Avg Packets/Bulk',
    'Fwd Blk Rate Avg': 'Fwd Avg Bulk Rate',
    'Bwd Byts/b Avg': 'Bwd Avg Bytes/Bulk',
    'Bwd Pkts/b Avg': 'Bwd Avg Packets/Bulk',
    'Bwd Blk Rate Avg': 'Bwd Avg Bulk Rate',
    'Subflow Fwd Pkts': 'Subflow Fwd Packets',
    'Subflow Fwd Byts': 'Subflow Fwd Bytes',
    'Subflow Bwd Pkts': 'Subflow Bwd Packets',
    'Subflow Bwd Byts': 'Subflow Bwd Bytes',
    'Init Fwd Win Byts': 'Init_Win_bytes_forward',
    'Init Bwd Win Byts': 'Init_Win_bytes_backward',
    'Fwd Act Data Pkts': 'act_data_pkt_fwd',
    'Fwd Seg Size Min': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min',
    'Label': 'Label',
}

In [ ]:
df2018_original.rename(columns=formatted_data, inplace=True)


In [ ]:
formatted_data = {
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packet': 'Total Fwd Packets',
    'Total Bwd packets': 'Total Backward Packets',
    'Total Length of Fwd Packet': 'Total Length of Fwd Packets',
    'Total Length of Bwd Packet': 'Total Length of Bwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    'Bwd PSH Flags': 'Bwd PSH Flags',
    'Fwd URG Flags': 'Fwd URG Flags',
    'Bwd URG Flags': 'Bwd URG Flags',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Packet Length Min': 'Min Packet Length',
    'Packet Length Max': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'SYN Flag Count': 'SYN Flag Count',
    'RST Flag Count': 'RST Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'URG Flag Count': 'URG Flag Count',
    'CWR Flag Count': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Count',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Average Packet Size': 'Average Packet Size',
    'Fwd Segment Size Avg': 'Avg Fwd Segment Size',
    'Bwd Segment Size Avg': 'Avg Bwd Segment Size',
    'Fwd Bytes/Bulk Avg': 'Fwd Avg Bytes/Bulk',
    'Fwd Packet/Bulk Avg': 'Fwd Avg Packets/Bulk',
    'Fwd Bulk Rate Avg': 'Fwd Avg Bulk Rate',
    'Bwd Bytes/Bulk Avg': 'Bwd Avg Bytes/Bulk',
    'Bwd Packet/Bulk Avg': 'Bwd Avg Packets/Bulk',
    'Bwd Bulk Rate Avg': 'Bwd Avg Bulk Rate',
    'Subflow Fwd Packets': 'Subflow Fwd Packets',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Subflow Bwd Packets': 'Subflow Bwd Packets',
    'Subflow Bwd Bytes': 'Subflow Bwd Bytes',
    'FWD Init Win Bytes': 'Init_Win_bytes_forward',
    'Bwd Init Win Bytes': 'Init_Win_bytes_backward',
    'Fwd Act Data Pkts': 'act_data_pkt_fwd',
    'Fwd Seg Size Min': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}

In [ ]:
df2018_corrected.rename(columns=formatted_data, inplace=True)

In [ ]:
df2017_original['Label'].value_counts()

BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Label, dtype: int64

In [ ]:
df2017_corrected['Label'].value_counts()

BENIGN                                    1582566
Portscan                                   159066
DoS Hulk                                   158468
DDoS                                        95144
Infiltration - Portscan                     71767
DoS GoldenEye                                7567
Botnet - Attempted                           4067
FTP-Patator                                  3972
DoS Slowloris                                3859
DoS Slowhttptest - Attempted                 3368
SSH-Patator                                  2961
DoS Slowloris - Attempted                    1847
DoS Slowhttptest                             1740
Web Attack - Brute Force - Attempted         1292
Botnet                                        736
Web Attack - XSS - Attempted                  655
DoS Hulk - Attempted                          581
DoS GoldenEye - Attempted                      80
Web Attack - Brute Force                       73
Infiltration - Attempted                       45


In [ ]:
df2018_original['Label'].value_counts()

Benign                      13484708
DDOS attack-HOIC              686012
DDoS attacks-LOIC-HTTP        576191
DoS attacks-Hulk              461912
Bot                           286191
FTP-BruteForce                193360
SSH-Bruteforce                187589
Infilteration                 161934
DoS attacks-SlowHTTPTest      139890
DoS attacks-GoldenEye          41508
DoS attacks-Slowloris          10990
DDOS attack-LOIC-UDP            1730
Brute Force -Web                 611
Brute Force -XSS                 230
SQL Injection                     87
Label                             59
Name: Label, dtype: int64

In [ ]:
df2018_corrected['Label'].value_counts()


BENIGN                                          59353486
DoS Hulk                                         1803160
DDoS-HOIC                                        1082293
FTP-BruteForce - Attempted                        298874
DDoS-LOIC-HTTP                                    289328
Botnet Ares                                       142921
SSH-BruteForce                                     94197
Infiltration - NMAP Portscan                       89374
DoS GoldenEye                                      22560
DoS Slowloris                                       8490
DoS GoldenEye - Attempted                           4301
DDoS-LOIC-UDP                                       2527
DoS Slowloris - Attempted                           2280
Botnet Ares - Attempted                              262
DDoS-LOIC-UDP - Attempted                            251
Infiltration - Communication Victim Attacker         204
Web Attack - Brute Force - Attempted                 137
Web Attack - Brute Force       

### handle infinity and -infinity

In [ ]:
df2017_original.isin([np.inf, -np.inf]).sum().sum()

4376

In [ ]:
df2017_corrected.isin([np.inf, -np.inf]).sum().sum()

10

In [ ]:
df2018_original.isin([np.inf, -np.inf]).sum().sum()

119498

In [ ]:
df2018_corrected.isin([np.inf, -np.inf]).sum().sum()


In [ ]:
df2017_original.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
df2017_corrected.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
df2018_original.replace([np.inf, -np.inf], np.nan, inplace=True)


In [ ]:
df2018_corrected.replace([np.inf, -np.inf], np.nan, inplace=True)


### Handle duplicate and missing value rows
- Delete deplicate rows
- Replace missing value with mean value

In [ ]:
def drop_unnecessary(df) :
    print(f'~~~~~~~ before drop {df.shape} ~~~~~~~')
    print(f'dataset contains {df.duplicated().sum()} of duplicated values')
    print(f'dataset contains {df.isna().sum().sum()} of missing values')
    df.drop_duplicates(keep="first", inplace=True)
    df.dropna(inplace=True)
    print(f'~~~~~~~ after drop {df.shape} ~~~~~~~')


In [ ]:
drop_unnecessary(df2017_original)

~~~~~~~ before drop (2830743, 79) ~~~~~~~
dataset contains 308381 of duplicated values
dataset contains 5734 of missing values
~~~~~~~ after drop (2520798, 79) ~~~~~~~


In [ ]:
drop_unnecessary(df2017_corrected)

~~~~~~~ before drop (2099976, 91) ~~~~~~~
dataset contains 0 of duplicated values
dataset contains 10 of missing values
~~~~~~~ after drop (2099971, 91) ~~~~~~~


In [ ]:
drop_unnecessary(df2018_original)

~~~~~~~ before drop (16233002, 80) ~~~~~~~
dataset contains 433253 of duplicated values
dataset contains 179219 of missing values
~~~~~~~ after drop (15708653, 80) ~~~~~~~


In [ ]:
df2017_original[['Flow Bytes/s', 'Flow Packets/s']].describe()

,Flow Bytes/s,Flow Packets/s
count,2.520798e+06,2.520798e+06
mean,1.410707e+06,4.729188e+04
std,2.657084e+07,2.026366e+05
min,-2.610000e+08,-2.000000e+06
25%,1.194308e+02,2.023326e+00
50%,3.715038e+03,6.974224e+01
75%,1.071429e+05,1.785714e+04
max,2.071000e+09,4.000000e+06


In [ ]:
df2017_corrected[['Flow Bytes/s', 'Flow Packets/s']].describe()

,Flow Bytes/s,Flow Packets/s
count,2.099971e+06,2.099971e+06
mean,4.660269e+05,1.931849e+04
std,3.977421e+06,1.068626e+05
min,0.000000e+00,2.500009e-02
25%,1.077790e+02,3.574474e+00
50%,3.864201e+03,7.369042e+01
75%,6.079603e+04,1.619433e+04
max,2.530000e+08,3.000000e+06


### Reclassify the data labels to ['Benign', 'Attack']

In [ ]:
def label_mapping(value):
    # if value == 'Benign':
    if (value == 'BENIGN') or ('Attempted' in value):
        return 0 #'Benigh'
    else:
        return 1 #'Attack'

In [ ]:
def reclassify_label(df):
    df['label_encoded'] = df['Label'].map(label_mapping)
    return df

In [ ]:
df2017_original = reclassify_label(df2017_original)

In [ ]:
df2017_corrected = reclassify_label(df2017_corrected)

In [ ]:
df2018_original = reclassify_label(df2018_original)

In [ ]:
df2018_corrected = reclassify_label(df2018_corrected)


### Encode Labels - Depreciated

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_label(df) :
    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['Label'])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(f'label mapping value: {le_name_mapping}')
    print(df['label_encoded'].value_counts())

In [ ]:
encode_label(df2017_original)

label mapping value: {'BENIGN': 0, 'Bot': 1, 'DDoS': 2, 'DoS GoldenEye': 3, 'DoS Hulk': 4, 'DoS Slowhttptest': 5, 'DoS slowloris': 6, 'FTP-Patator': 7, 'Heartbleed': 8, 'Infiltration': 9, 'PortScan': 10, 'SSH-Patator': 11, 'Web Attack � Brute Force': 12, 'Web Attack � Sql Injection': 13, 'Web Attack � XSS': 14}
0     2096134
4      172846
2      128016
10      90819
3       10286
7        5933
6        5385
5        5228
11       3219
1        1953
12       1470
14        652
9          36
13         21
8          11
Name: label_encoded, dtype: int64


### Prepare for algorithms

In [ ]:
df2017_original_y = df2017_original['label_encoded'].values
#remove unneccessary and duplicate column
df2017_original_X = df2017_original.drop(['Destination Port', 'Fwd Header Length.1', 'label_encoded', 'Label'], axis=1)
data = pd.DataFrame(data=df2017_original_X)

data['label_encoded'] = df2017_original_y
data['label_encoded'].value_counts()

0    2095057
1     425741
Name: label_encoded, dtype: int64

In [ ]:
data.to_parquet('data_2017_original.parquet', index = False, compression=None)

In [ ]:
pd.read_parquet('results/data_2017_original.parquet', engine="fastparquet")

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded
0,3,2,0,12,0,6,6,6.0,0.00000,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,109,1,1,6,6,6,6,6.0,0.00000,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,52,1,1,6,6,6,6,6.0,0.00000,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,34,1,1,6,6,6,6,6.0,0.00000,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,3,2,0,12,0,6,6,6.0,0.00000,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520793,32215,4,2,112,152,28,28,28.0,0.00000,76,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2520794,324,2,2,84,362,42,42,42.0,0.00000,181,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2520795,82,2,1,31,6,31,0,15.5,21.92031,6,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
2520796,1048635,6,2,192,256,32,32,32.0,0.00000,128,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [ ]:
df2017_corrected.head(3)

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category,label_encoded
0,1,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,...,1.610540e+07,4.988048e+05,16399772,15375229,-1,-1,112740690,BENIGN,-1,0
1,2,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,...,1.610543e+07,4.987937e+05,16399782,15375263,-1,-1,112740560,BENIGN,-1,0
2,3,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,545,...,1.221036e+07,6.935824e+06,20757030,5504997,-1,-1,0,BENIGN,-1,0


In [ ]:
df2017_corrected_y = df2017_corrected['label_encoded'].values
#remove unneccessary and duplicate column
df2017_corrected_X = df2017_corrected.drop(['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst Port', 'Dst IP', 'Protocol', 'Timestamp', 'label_encoded', 'Label'], axis=1)
data = pd.DataFrame(data=df2017_corrected_X)

data['label_encoded'] = df2017_corrected_y
data['label_encoded'].value_counts()

0    1594540
1     505431
Name: label_encoded, dtype: int64

In [ ]:
data.to_parquet('/content/onedrive/data_2017_corrected.parquet', index = False, compression=None)

In [ ]:
pd.read_parquet('results/data_2017_corrected.parquet', engine="fastparquet")

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Attempted Category,label_encoded
0,112740690,32,16,6448,1152,403,0,201.500000,204.724205,72,...,343,1.610540e+07,4.988048e+05,16399772,15375229,-1,-1,112740690,-1,0
1,112740560,32,16,6448,5056,403,0,201.500000,204.724205,316,...,285,1.610543e+07,4.987937e+05,16399782,15375263,-1,-1,112740560,-1,0
2,113757377,545,0,0,0,0,0,0.000000,0.000000,0,...,19,1.221036e+07,6.935824e+06,20757030,5504997,-1,-1,0,-1,0
3,91997219,388,0,37151,0,227,37,95.750000,55.785320,0,...,16,1.319764e+07,5.826905e+06,19776791,5817470,-1,-1,0,-1,0
4,66966070,6,6,288,288,48,48,48.000000,0.000000,48,...,1968172,6.497443e+07,0.000000e+00,64974431,64974431,-1,-1,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099971,5074745,7,6,582,1204,582,0,83.142857,219.975323,602,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,5074745,-1,0
2099972,209,2,2,70,230,35,35,35.000000,0.000000,115,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,-1,0
2099973,116281383,24,21,699,5411,322,0,29.125000,79.231808,1448,...,23012,9.639210e+06,1.321437e+06,10024910,5443180,-1,-1,116281383,-1,0
2099974,149,2,2,72,104,36,36,36.000000,0.000000,52,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,-1,0


In [ ]:
df2018_origingal_y = df2018_original['label_encoded'].values
#remove unneccessary and duplicate column
df2018_original_X = df2018_original.drop(['Destination Port', 'Protocol', 'Timestamp', 'label_encoded', 'Label'], axis=1)
data = pd.DataFrame(data=df2018_original_X)

data['label_encoded'] = df2018_origingal_y
data['label_encoded'].value_counts()

0    13355246
1     2353407
Name: label_encoded, dtype: int64

In [ ]:
# data = pd.to_numeric(data, errors='coerce')
data = data.apply(pd.to_numeric, errors='coerce')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15708653 entries, 0 to 16233001
Data columns (total 77 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Flow Duration                float64
 1   Total Fwd Packets            float64
 2   Total Backward Packets       float64
 3   Total Length of Fwd Packets  float64
 4   Total Length of Bwd Packets  float64
 5   Fwd Packet Length Max        float64
 6   Fwd Packet Length Min        float64
 7   Fwd Packet Length Mean       float64
 8   Fwd Packet Length Std        float64
 9   Bwd Packet Length Max        float64
 10  Bwd Packet Length Min        float64
 11  Bwd Packet Length Mean       float64
 12  Bwd Packet Length Std        float64
 13  Flow Bytes/s                 float64
 14  Flow Packets/s               float64
 15  Flow IAT Mean                float64
 16  Flow IAT Std                 float64
 17  Flow IAT Max                 float64
 18  Flow IAT Min                 float64
 19

In [ ]:
data.to_parquet('data_2018_original.parquet', index = False, compression=None, engine='fastparquet')

In [ ]:
pd.read_parquet('data_2018_original.parquet', engine="fastparquet")

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded
0,141385.0,9.0,7.0,553.0,3773.0,202.0,0.0,61.444444,87.534438,1460.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,281.0,2.0,1.0,38.0,0.0,38.0,0.0,19.000000,26.870058,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,279824.0,11.0,15.0,1086.0,10527.0,385.0,0.0,98.727273,129.392497,1460.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,132.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,274016.0,9.0,13.0,1285.0,6141.0,517.0,0.0,142.777778,183.887722,1460.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15708648,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
15708649,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
15708650,732728.0,2.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
15708651,22.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
df2018_corrected_y = df2018_corrected['label_encoded'].values
#remove unneccessary and duplicate column
df2018_corrected_X = df2018_corrected.drop(['label_encoded', 'Label'], axis=1)
data = pd.DataFrame(data=df2018_corrected_X)

data['label_encoded'] = df2018_corrected_y
data['label_encoded'].value_counts()

In [ ]:
data.to_parquet('data_2018_corrected.parquet', index = False, compression=None, engine='fastparquet')

In [ ]:
pd.read_parquet('data_2018_corrected.parquet', engine="fastparquet")


### Data Resampling
- under sampling for majority
- over sampling for minoirty

In [ ]:
from imblearn.under_sampling import RandomUnderSampler


def under_sampling(X_train, y_train):
    unique_values, value_counts = np.unique(y_train, return_counts=True)
    print(f'~~~Before UnderSampling: {value_counts}~~~')
    rus = RandomUnderSampler(random_state=42)
    X_train, y_train = rus.fit_resample(X_train, y_train)
    unique_values, value_counts = np.unique(y_train, return_counts=True)
    print(f'~~~After UnderSampling: {value_counts}')
    return X_train, y_train

In [ ]:
from imblearn.over_sampling import SMOTE

def over_sampling(X_train, y_train):
    unique_values, value_counts = np.unique(y_train, return_counts=True)
    print(f'~~~Before OverSampling: {value_counts}~~~')
    sm = SMOTE(random_state=2)
    X_train, y_train = sm.fit_resample(X_train, y_train.ravel())
    unique_values, value_counts = np.unique(y_train, return_counts=True)
    print(f'~~~After OverSampling: {value_counts}~~~')
    return X_train, y_train


In [ ]:
X_train_resample, y_train_resample = under_sampling(df2017_original_X, df2017_original_y)

data_resampling = pd.DataFrame(data=X_train_resample)
data_resampling['label_encoded'] = y_train_resample
data_resampling['label_encoded'].value_counts()

~~~Before UnderSampling: [2095057  425741]~~~
~~~After UnderSampling: [425741 425741]


0    425741
1    425741
Name: label_encoded, dtype: int64

In [ ]:
data_resampling.to_parquet('data_2017_original_resampling.parquet', index = False)

In [ ]:
pd.read_parquet('results/data_2017_original_resampling.parquet', engine="fastparquet")

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded
0,858591,2,2,72,124,36,36,36.000000,0.000000,62,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
1,60706,2,2,70,290,35,35,35.000000,0.000000,145,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
2,257846,1,1,46,208,46,46,46.000000,0.000000,208,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
3,3,2,0,0,0,0,0,0.000000,0.000000,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
4,1559160,35,42,2622,7038,408,0,74.914286,105.280961,976,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851745,11512204,8,5,326,11632,326,0,40.750000,115.258405,10184,...,32,892.0,0.0,892,892,6507197.0,0.0,6507197,6507197,1
851746,11513325,5,5,471,3525,471,0,94.200000,210.637604,2077,...,32,918.0,0.0,918,918,6508582.0,0.0,6508582,6508582,1
851747,11509201,7,6,314,11632,314,0,44.857143,118.680845,5792,...,32,899.0,0.0,899,899,6503248.0,0.0,6503248,6503248,1
851748,11509095,8,5,369,11632,369,0,46.125000,130.461201,10184,...,32,914.0,0.0,914,914,6504954.0,0.0,6504954,6504954,1


In [ ]:
X_train_resample, y_train_resample = under_sampling(df2017_corrected_X, df2017_corrected_y)

data_resampling = pd.DataFrame(data=X_train_resample)
data_resampling['label_encoded'] = y_train_resample
data_resampling['label_encoded'].value_counts()

~~~Before UnderSampling: [1594540  505431]~~~
~~~After UnderSampling: [505431 505431]


0    505431
1    505431
Name: label_encoded, dtype: int64

In [ ]:
data_resampling.to_parquet('/content/onedrive/data_2017_corrected_resampling.parquet', index = False)

In [ ]:
pd.read_parquet('data_2017_corrected_resampling.parquet', engine="fastparquet")


,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Attempted Category,label_encoded
0,31783840,18,12,857,9046,373,0,47.611111,105.292388,2836,...,195460,3.155016e+07,0.000000e+00,31550155,31550155,-1,-1,31783840,-1,0
1,23681,1,1,58,139,58,58,58.000000,0.000000,139,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,-1,0
2,266,2,2,64,242,32,32,32.000000,0.000000,121,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,-1,0
3,60955642,21,22,6472,12096,3228,0,308.190476,923.323758,1494,...,87887,9.927016e+06,4.389169e+05,10152070,9039441,-1,-1,60955642,-1,0
4,803,2,2,76,190,38,38,38.000000,0.000000,95,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034815,656477,11,5,1041,11595,347,0,94.636364,162.083480,7240,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,656477,-1,1
1034816,103680007,13,6,1472,11632,368,0,113.230769,176.781482,4344,...,3207,3.426969e+07,5.022459e+07,92263198,5000791,-1,-1,103680007,-1,1
1034817,11464350,9,5,588,11632,588,0,65.333333,196.000000,7288,...,749,6.423318e+06,0.000000e+00,6423318,6423318,-1,-1,11464350,-1,1
1034818,177006,8,8,322,11595,322,0,40.250000,113.844192,7240,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,177006,-1,1


In [ ]:
X_train_resample, y_train_resample = under_sampling(df2018_original_X, df2018_origingal_y)

data_resampling = pd.DataFrame(data=X_train_resample)
data_resampling['label_encoded'] = y_train_resample
data_resampling['label_encoded'].value_counts()

~~~Before UnderSampling: [13355246  2353407]~~~
~~~After UnderSampling: [2353407 2353407]


0    2353407
1    2353407
Name: label_encoded, dtype: int64

In [ ]:
data_resampling = data_resampling.apply(pd.to_numeric, errors='coerce')

In [ ]:
data_resampling.to_parquet('data_2018_original_resampling.parquet', index = False)

In [ ]:
pd.read_parquet('data_2018_original_resampling.parquet', engine="fastparquet")

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded
0,419026.0,2.0,0.0,44.0,0.0,22.0,22.0,22.000000,0.000000,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,4710.0,3.0,0.0,41.0,0.0,41.0,0.0,13.666667,23.671361,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,57774.0,2.0,2.0,70.0,336.0,35.0,35.0,35.000000,0.000000,168.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1742.0,1.0,1.0,37.0,222.0,37.0,37.0,37.000000,0.000000,222.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,6.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4706809,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4706810,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4706811,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4706812,22.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


### Special Handle for 2018 Improved datases - Too large to run through above preprocessing

In [ ]:
# function that creates random sample
def random_sampling(df, n):
    random_indices = np.random.choice(df.index, replace=False, size=n)
    random_sample = df.loc[random_indices]
    return random_sample

In [ ]:
def preprocessing(df, name, is_resampling = False):

    df = reclassify_label(df)

    df0 = df[df['label_encoded'] == 0]
    df1 = df[df['label_encoded'] == 1]

    size_df0 = df0.shape[0]
    size_df1 = df1.shape[0]

    if is_resampling:
        df0 = random_sampling(df0, size_df1)
    else:
        df0 = random_sampling(df0, int(size_df0*0.1))

    df_temp = pd.concat([df0, df1])
    # print(f'df has {df.isin([np.inf, -np.inf]).sum().sum()} infinity value')
    df_temp.replace([np.inf, -np.inf], np.nan, inplace=True)
    # print(f'dataset contains {df.duplicated().sum()} of duplicated values')
    # print(f'dataset contains {df.isna().sum().sum()} of missing values')
    df_temp.drop_duplicates(keep="first", inplace=True)
    df_temp.dropna(inplace=True)
    df_temp.rename(columns=formatted_data, inplace=True)



    y = df_temp['label_encoded'].values
    #remove unneccessary and duplicate column
    X = df_temp.drop(['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst Port', 'Dst IP', 'Protocol', 'Timestamp', 'label_encoded', 'Attempted Category', 'Label'], axis=1)


    data = pd.DataFrame(data=X)

    data['label_encoded'] = y
    return data

In [ ]:
d0_2018_c = df2018_corrected #the first row portion is already imported so we will just copy that
d0_2018_c = preprocessing(d0_2018_c, 'd0')

del df2018_corrected

d1_2018_c = pd.read_parquet('../resources/Improved/CSECICIDS2018_improved/Friday-16-02-2018.parquet')
d1_2018_c = preprocessing(d1_2018_c, 'd1')

d2_2018_c = pd.read_parquet('../resources/Improved/CSECICIDS2018_improved/Friday-23-02-2018.parquet')
d2_2018_c = preprocessing(d2_2018_c, 'd2')

d3_2018_c = pd.read_parquet('../resources/Improved/CSECICIDS2018_improved/Thursday-01-03-2018.parquet')
d3_2018_c = preprocessing(d3_2018_c, 'd3')

d4_2018_c = pd.read_parquet('../resources/Improved/CSECICIDS2018_improved/Thursday-15-02-2018.parquet')
d4_2018_c = preprocessing(d4_2018_c, 'd4')

d5_2018_c = pd.read_parquet('../resources/Improved/CSECICIDS2018_improved/Thursday-22-02-2018.parquet')
d5_2018_c = preprocessing(d5_2018_c, 'd5')

d6_2018_c = pd.read_parquet('../resources/Improved/CSECICIDS2018_improved/Tuesday-20-02-2018.parquet')
d6_2018_c = preprocessing(d6_2018_c, 'd6')

d7_2018_c = pd.read_parquet('../resources/Improved/CSECICIDS2018_improved/Wednesday-14-02-2018.parquet')
d7_2018_c = preprocessing(d7_2018_c, 'd7')

d8_2018_c = pd.read_parquet('../resources/Improved/CSECICIDS2018_improved/Wednesday-21-02-2018.parquet')
d8_2018_c = preprocessing(d8_2018_c, 'd8')

d9_2018_c = pd.read_parquet('../resources/Improved/CSECICIDS2018_improved/Wednesday-28-02-2018.parquet')
d9_2018_c = preprocessing(d9_2018_c, 'd9')


In [ ]:
df2018_corrected = pd.concat([d0_2018_c, d1_2018_c, d2_2018_c, d3_2018_c, d4_2018_c, d5_2018_c, d6_2018_c, d7_2018_c, d8_2018_c, d9_2018_c], ignore_index=True)


In [ ]:
df2018_corrected.shape


In [ ]:
df2018_corrected.to_parquet(f'/content/onedrive/data_2018_corrected_sampling.parquet', index = False, compression=None, engine='fastparquet')

In [ ]:
d0_2018_c = df2018_corrected #the first row portion is already imported so we will just copy that
d0_2018_c = preprocessing(d0_2018_c, 'd0', True)

del df2018_corrected

d1_2018_c = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Friday-16-02-2018.parquet')
d1_2018_c = preprocessing(d1_2018_c, 'd1', True)

d2_2018_c = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Friday-23-02-2018.parquet')
d2_2018_c = preprocessing(d2_2018_c, 'd2', True)

d3_2018_c = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Thursday-01-03-2018.parquet')
d3_2018_c = preprocessing(d3_2018_c, 'd3', True)

d4_2018_c = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Thursday-15-02-2018.parquet')
d4_2018_c = preprocessing(d4_2018_c, 'd4', True)

d5_2018_c = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Thursday-22-02-2018.parquet')
d5_2018_c = preprocessing(d5_2018_c, 'd5', True)

d6_2018_c = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Tuesday-20-02-2018.parquet')
d6_2018_c = preprocessing(d6_2018_c, 'd6', True)

d7_2018_c = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Wednesday-14-02-2018.parquet')
d7_2018_c = preprocessing(d7_2018_c, 'd7', True)

d8_2018_c = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Wednesday-21-02-2018.parquet')
d8_2018_c = preprocessing(d8_2018_c, 'd8', True)

d9_2018_c = pd.read_parquet('/content/onedrive/Improved/CSECICIDS2018_improved/Wednesday-28-02-2018.parquet')
d9_2018_c = preprocessing(d9_2018_c, 'd9', True)


In [ ]:
df2018_corrected_r = pd.concat([d0_2018_c, d1_2018_c, d2_2018_c, d3_2018_c, d4_2018_c, d5_2018_c, d6_2018_c, d7_2018_c, d8_2018_c, d9_2018_c], ignore_index=True)


In [ ]:
df2018_corrected_r.shape


In [ ]:
df2018_corrected_r.to_parquet(f'/content/onedrive/data_2018_corrected_sampling_resampling.parquet', index = False, compression=None, engine='fastparquet')
